# <b>Processing FISHnCHIPs images of the mouse brain </b>

FISHnCHIPs data processing with the aim of obtaining the spatial positions of the cells and the fluorescence intensity of the cell mask (cell-by-module intensity matrix). Clustering of the cell-by-module intensity matrix can be used to determine cell types. 

## *Preparation of schema file*

<p>The schema file contains information on the dye and hyb number for each gene module imaged.</p>
<p>The schema file should be in csv format. Refer to <font color=sandybrown>template_schema.xlsx</font> in the <font color=royalblue>config_yaml</font> folder for reference.</p>

## *Preparation of yaml file*

Create a yaml file or use the <font color=sandybrown>template_yaml.yaml</font> file in the <font color=royalblue>config_yaml</font> folder as template.

*In the yaml file, include the following information:*

* **mainpath**: The file path where DAPI and hyb image TIF file are stored
* **schema**: The file path where the schema csv file is located. Schema file should contain the cell type to analyse and its corresponding dye
* **dapiname**: Prefix name of dapi file
* **fovs**: Number of field of views (FOVs) to process
* **fov_x**: Number of FOVs along x-axis
* **fov_y**: Number of FOVs along y-axis
* **overlap_in_px**: Number of pixel that overlaps between each FOV
* **background**: Default "bleach"; Type of background image used to offset background noise
* **remove_background**: Default 'subtract'; How background noise is removed
* **show_dapiseg**: Boolean; Whether to run segmentation function
* **segmentation**: Select segmentation method; 'cellpose' or 'watershed'
* **segment_mode**: 'Cytoplasm', 'Cytoplasm2', 'Cytoplasm2_Omnipose', 'Nuclei' for cellpose
* **cellsize**: Cell size in μm
* **flow_threshold**: Default 0.4; Increase threshold if cellpose is not returning as many ROIs as you’d expect.
* **cellprob_threshold**: Default 0.0; Decrease this threshold if cellpose is not returning as many ROIs as you’d expect.
* **mask_dilate_factor**: Amount of dilation applied to cell mask
* **filtermask**: Boolean; Whether to remove small masks
* **filtermask_size**: Minimum mask size
* **anchor_name**: Prefix of hyb image to segment e.g.'Cy7_3_'
* **anchor_celltype**: Celltype label of the hyb image e.g.'CAF_1'


*Configuration for watershed segmentation*

* **fusedpath**: File path of fused image to determine the minimum and maximum intensity at a certain percentile
* **cutoff_threshold**: Threshold value which is used to classify the pixel values into foreground and background classes, creating a binary image
* **opening_threshold**: Number of iterations of erosion and dilation that the image goes through
* **kernel_size**: Size of kernel that slides through the image which will determine how much the image is eroded or dilated

## *Installing and imprting relevant packages*

Prior to importing the packages and functions, please install the FishnChips package with the following installation command: 
<p><font color=blue>pip install -i https://test.pypi.org/simple/ FishnChips</font> </p>
<p>This tutorial uses cellpose alogorithm for cellular segmentation. Install cellpose using the following command:</p>
<p><font color=blue>pip install cellpose</font></p>
<p>Scikit-image is used for image processing in this tutorial. Install scikit-image using the following command:</p>
<p><font color=blue>pip install -U scikit-image</font></p>

In [15]:
import os
import yaml
import shutil

import pandas as pd
import matplotlib.pyplot as plt

from multiprocessing import Pool
from itertools import repeat
import itertools
#from FishnChips (use this line when package has been uploaded)
from scripts import FISHnCHIPsImages, segmentationFunctions 

<div></div>

## *Segment DAPI image*

### 1) Import configuration from yaml file

Fetch the yaml file from its directory and define the FOVs to analyse. All FOVs will be analysed if no FOVs were specified.

In [2]:
# Input setting file ---------------------------------------------------------------------------------------------------------
yml_file = r'./config_yaml/register_maxshift50params.yaml'
fov_list = None # User-defined FOVs to analyse: e.g. fov_list = ['010', '012', '059']; if None, analyse all fovs by default

<p>Load configuration from the yaml file into the variable <font color=purple>user_dict</font> as a dictionary. </p>

<p>Create output path based on segmentation method and parameters and generate folder for storing output if it does not exist. </p>Saves a copy of the configuration used in the output folder as a yaml file.

In [3]:
### Parsing the .yaml file and create output folder ------------------------------------------------------------------------
with open(yml_file) as file:
    user_dict= yaml.load(file, Loader=yaml.FullLoader)
if user_dict['segmentation']=='cellpose':
    output_path = os.path.join('./output_files/', 'cellpose_%s_cs%s_k%s_filt%i_DAPI/' % (str(user_dict['segment_mode']), str(user_dict['cellsize']), 
                                                                                  str(user_dict['mask_dilate_factor']), user_dict['filtermask_size']))
elif user_dict['segmentation'] == 'watershed':
    output_path = os.path.join('./watershed/')
print(output_path)
if not os.path.exists(output_path):
    os.makedirs(output_path)
    shutil.copyfile(yml_file, output_path + "params.yaml")

./output_files/cellpose_Nuclei_cs15_k2_filt0_DAPI/


### 2) Initialize parameters for segmentation

segmentDAPI: boolean input, True to run segmentation on DAPI images

In [4]:
# Parameters for segment DAPI -------------------------------------------------------------------------------------------------
segmentDAPI = True

### 3) Cell segmentation for every FOV image

<p>Ignore this section if segmentation is not required for your usage (<font color=maroon>segmentDAPI</font> = FALSE).</p>
<p>Segment all FOVs if no FOVs are specified. Segment FOVs using the <font color=purple>segment_dapi_one_fov</font> function from <font color=forestgreen>FISHnCHIPsImages</font>.</p>
<p>Multiprocessing of the segmentation is conducted to conserve time. Please adjust the number of processors to use according to your machine specifications. For this case, 8 processors were used for multiprocessing.</p>
<p>The function returns the list of cell masks for the analyzed FOVs and saves each segmented image in .tif format (image on the left).</p> 
<p>An overlapped image of the DAPI image and cell mask is also saved in .jpg format, showing the number of cell masks (image on the right).</p>

<table><tr>
    <td> <img src="./assets/DAPI_rm389_fov_0.jpg" width="380" /> <p><center><b>Segmented image in tif format</b></center></p></td>
<td> <img src="./assets/dapi_overlay_mask_fov_0.jpg" width="510" /> <p><center><b>Segmented image and cell mask overlay in jpg format</b></center></p> </td>
</tr></table>

In [5]:
### Segment DAPI ----------------------------------------------------------------------------------------------------------
if segmentDAPI:
    ndigits = len(str(user_dict['fovs'])) # 2, 3 etc 
    if fov_list is None:
        fov_list = [str(x).zfill(ndigits) for x in range(user_dict['fovs'])]

    inputs = zip(repeat(user_dict), repeat(output_path), fov_list)
    with Pool(8) as p:
        p.starmap(FISHnCHIPsImages.segment_dapi_one_fov, inputs)
    print("Segementation for all fovs......Done!")

Segementation for all fovs......Done!


### 4) Obtaining mask positions from stitched and segmented image

<p>Create list of FOVs containing all FOVs that constitutes of the whole stitched image.</p>
<p>Create a list of filenames that starts with the same prefix in the output directory.</p>

In [6]:
### Scan all mask images for all FOVs and parse mask positions -------------------------------------------------------------
fov_list_full = [str(x).zfill(ndigits) for x in range(user_dict['fovs'])]
dapi_prefix = 'DAPI_rm389_fov'
prefix = 'dilateMask_fov'
prefixed = [filename for filename in os.listdir(output_path) if filename.startswith(prefix)]

<p>Using multiprocessing, the x and y coordinates of each centroid are obtained from the cell mask of each FOV image using the <font color=purple>get_centroids</font> from <font color=forestgreen>segmentationFunctions</font> and outputs the results as a csv file (<font color=mediumpurple>'centroids_in_FOVs.csv'</font>).</p>
<p>After obtaining the x and y coordinates of each centroid for each FOV image, the overall position of the centroid/mask in the fused image (after stitching all FOV images together) is determined using the <font color=purple>get_mask_positions_inFuse</font> from <font color=forestgreen>segmentationFunctions</font> and outputs the result as a csv file (<font color=mediumpurple>'centroids_in_Fuse.csv'</font>).</p>
<p>Lastly, the cell mask positions are plotted, visualized and saved as a jpg image (<font color=mediumpurple>'cells_scatterplt.jpg'</font>).</p>

<img src="./assets/cells_scatterplt.jpg" width="380" /> <p><center><b>Scatterplot of cell masks</b></center></p></td>


In [7]:
if len(prefixed) == len(fov_list_full):
    with Pool(8) as p:
        result = p.starmap(segmentationFunctions.get_centroids, zip(repeat(output_path), repeat(prefix), repeat(dapi_prefix), fov_list_full))

    df_centroids = pd.concat(result, axis=0)
    df_centroids.to_csv(os.path.join(output_path, 'centroids_in_FOVs.csv'))
    print(df_centroids)

    centroid_list = df_centroids['Centroid'].to_list()
    list_fov = df_centroids['FOV'].to_list()
    list_x = [t[0] for t in centroid_list]
    list_y = [t[1] for t in centroid_list]

    mask_positions = segmentationFunctions.get_mask_positions_inFuse(list_fov, list_x, list_y, user_dict['fov_x'], user_dict['fov_y'])
    mask_positions.to_csv(os.path.join(output_path, 'centroids_in_Fuse.csv'))
    print(mask_positions)

    plt.figure(figsize = (6,6))
    plt.scatter(mask_positions['x'], mask_positions['y'], s=0.5)
    plt.gca().invert_yaxis()
    plt.gca().set_aspect('equal')
    plt.tight_layout()
    plt.savefig(os.path.join(output_path, 'cells_scatterplt.jpg'), 
                bbox_inches='tight', dpi = 120)
    plt.close('all')

else:
    print('%i out of %i FOVs have been segmented. \n \
          Please segment all FOVs before parsing the cells locations in the fused image.' % (len(prefixed), len(fov_list_full)))

                                     Centroid  label FOV  Area  \
0      (78.78378378378379, 2.027027027027027)      1   0    37   
1      (929.4961832061068, 9.903307888040713)      2   0   393   
2     (972.7708333333334, 1.8541666666666667)      3   0    48   
3    (1082.6363636363637, 2.2954545454545454)      4   0    44   
4                             (1265.8, 4.975)      5   0    80   
..                                        ...    ...  ..   ...   
948    (967.569105691057, 1263.5528455284552)    949   3   123   
949   (1236.393103448276, 1262.7586206896551)    950   3   145   
950             (1263.75, 1266.0131578947369)    951   3    76   
951                         (1077.5, 1266.22)    952   3    50   
952                (73.5, 1266.5740740740741)    953   3    54   

     DAPI_Mean_inten  DAPI_Max_inten  DAPI_Median_inten  DAPI_sum_inten  \
0         424.783784             548              408.5           15717   
1         569.763359             783              539.0  

<p> </p>

## *Subtract bleach from hyb images and register shifts to DAPI*

### 1) Import configuration from yaml file

Fetch the yaml file from its directory and define the FOVs and celltypes to analyse. 
All FOVs and celltypes will be analysed if none were specified.

In [8]:
# Input setting file ---------------------------------------------------------------------------------------------------------
yml_file = r'./config_yaml/register_maxshift50params.yaml'

fov_list = None # User-defined FOVs to analyse: e.g. fov_list = ['010', '012', '059']; if None, analyse all fovs by default
celltype_list = None

ndigits = len(str(user_dict['fovs']))
if fov_list is None:
    fov_list = [str(x).zfill(ndigits) for x in range(user_dict['fovs'])] # for pool 
if celltype_list is None:
    image_obj_0 = FISHnCHIPsImages.yml_class(user_dict, output_path)
    celltype_list = image_obj_0.imagename_dict.keys()

<p>Load configuration from the yaml file into the variable <font color=purple>user_dict</font> as a dictionary. </p>

<p>Create output path based on segmentation method and parameters and generate folder for storing output if it does not exist. </p>Saves a copy of the configuration used in the output folder as a yaml file.

In [10]:
### Parsing the .yaml file and create output folder ------------------------------------------------------------------------
with open(yml_file) as file:
    user_dict= yaml.load(file, Loader=yaml.FullLoader)
output_path = os.path.join('./output_files/', 'bleach_subtract/' )
print(output_path)
if not os.path.exists(output_path):
    os.makedirs(output_path)
    shutil.copyfile(yml_file, output_path + "params.yaml")

./output_files/bleach_subtract/


### 2) Subtract bleach from each FOV image and register any shifts in image to DAPI

<p>Using multiprocessing, the intensity of the bleached image is subtracted from the hyb images to maximize visibility of the cells and any shifts in the hyb image is registered to the DAPI image using the <font color=purple>subtract_one_image</font> function from <font color=forestgreen>FISHnCHIPsImages</font>. </p>
<p>The function outputs the bleach-subtracted hyb images in .tif format.</p>

In [11]:
### Subtract images
for celltype in celltype_list:
    inputs = zip(repeat(user_dict), repeat(output_path), repeat(celltype), fov_list)
    with Pool(8) as p:
        p.starmap(FISHnCHIPsImages.subtract_one_image, inputs)

<p></p>

## *Segment DAPI and register all hybs to DAPI (only if all FOVs of DAPI and hyb images are ready)*

<p>A combination of all the previous functionalities, including segmentation, bleach subtraction, registration of shifts to DAPI, mask position and intensity information, when all DAPI and hyb image files are ready. </p>

### 1) Import configuration from yaml file

<p>As usual, fetch yaml file from its directory and load configuration from the yaml file into the variable <font color=purple>user_dict</font> as a dictionary. </p>
<p>Create output directory if it does not exist and save a copy of the configurations used the current data analysis run in the directory as a yaml file.</p>

In [12]:
### User inputs -----------------------------------------------------------------------------------------------------------------------
yml_file = r'./config_yaml/register_maxshift50params.yaml'

In [13]:
### Parsing the .yaml file and create output folder ------------------------------------------------------------------------
with open(yml_file) as file:
    user_dict= yaml.load(file, Loader=yaml.FullLoader)
if user_dict['segmentation']=='cellpose':
    output_path = os.path.join('./output_files/', 'cellpose_%s_cs%s_k%s_filt%i/' % (str(user_dict['segment_mode']), str(user_dict['cellsize']), 
                                                                                  str(user_dict['mask_dilate_factor']), user_dict['filtermask_size']))
elif user_dict['segmentation'] == 'watershed':
    output_path = os.path.join('./watershed/')
print(output_path)
if not os.path.exists(output_path):
    os.makedirs(output_path)
    shutil.copyfile(yml_file, output_path + "params.yaml")

./output_files/cellpose_Nuclei_cs15_k2_filt0/


<p>Create list of FOVs containing all FOVs that constitutes of the whole stitched image.</p>
<p>Similar to the <font color=purple>segment_dapi_one_fov</font> function, the <font color=purple>segment_one_fov</font> function from <font color=forestgreen>FISHnCHIPsImages</font> segments each FOV of the available hyb images and returns the list of cell masks for each FOV, which is merged to obtain the full list of cell masks for all FOVs. The <font color=purple>segment_one_fov</font> function also includes the bleach subtraction and shift registration to DAPI process mentioned in the previous workflows.</p>
<p>Finally, the list of cell masks from all FOVs is converted into a <font color=lightcoral>Cell_masks</font> object which will be used for spatial and mask intensity analysis in the following part.</p>

In [16]:
# Segmentation
ndigits = len(str(user_dict['fovs']))
fov_list = [str(x).zfill(ndigits) for x in range(user_dict['fovs'])] # for pool 
inputs = zip(repeat(user_dict), repeat(output_path), fov_list)
with Pool(8) as p:
    list_all_cellmask = p.starmap(FISHnCHIPsImages.segment_one_fov, inputs)
merged = list(itertools.chain.from_iterable(list_all_cellmask)) # combine lists of cellmask from all workers

All_masks = FISHnCHIPsImages.Cell_masks(merged) # object of all cell masks
print("Segementation for all fovs......Done!")

Segementation for all fovs......Done!


<p>Finally, the mean, median, maximum and summation of mask intensity are calculated from the <font color=lightcoral>Cell_masks</font> object containing all cell masks from all FOVs using the <font color=purple>get_mask_intensity_matrix</font> function from <font color=forestgreen>segmentationFunctions</font>.</p>
<p>Other information of the cell masks such as the list of FOVs analysed, cell types identified, mask intensity, area of masks and mask spatial positions are saved as a csv file.</p>
<p>To obtain the spatial position in the context where all FOVs are fused into a single image, the FOV and spatial position of each cell mask in their respective FOVs is extracted using the <font color=purple>get_mask_positions_inFOV</font> function from <font color=forestgreen>segmentationFunctions</font>. The output from the <font color=purple>get_mask_positions_inFOV</font> function was fed into the <font color=purple>get_mask_positions_inFuse</font> function to obtain the overall positions of the cell mask in the fused image, saving the spatial positions of the cell masks as a csv file.</p>
<p>The cell masks are visualized using a plot which is saved as a jpg file.</p>

In [17]:
# Parse cells information
df_mean, df_sum, df_max, df_median = segmentationFunctions.get_mask_intensity_matrix(All_masks)
df_mean.to_csv(output_path + 'cellbits_matrix_mean_inten.csv')
df_sum.to_csv(output_path + 'cellbits_matrix_sum_inten.csv')
df_max.to_csv(output_path + 'cellbits_matrix_max_inten.csv')
df_median.to_csv(output_path + 'cellbits_matrix_median_inten.csv')

mask_info = segmentationFunctions.get_mask_info(All_masks)
mask_info.to_csv(output_path + 'all_masks_data.csv')
list_fov, list_x, list_y = segmentationFunctions.get_mask_positions_inFOV(mask_info)
image_size = 2048-int(user_dict['overlap_in_px'])*2
mask_positions = segmentationFunctions.get_mask_positions_inFuse(list_fov, list_x, list_y, user_dict['fov_x'], user_dict['fov_y'], image_size = image_size)
mask_positions.to_csv(output_path + 'cells_positions.csv')
print("Parsing cell masks intensity......Done! ")

plt.figure(figsize = (6,6))
plt.scatter(mask_positions['x'], mask_positions['y'], s=0.5)
plt.gca().invert_yaxis()
plt.gca().set_aspect('equal')
plt.tight_layout()
plt.savefig(os.path.join(output_path, 'cells_positions_scatter.jpg'), 
            bbox_inches='tight', dpi = 120)
plt.close('all')

row: 0 col: 0 fov: 2
row: 0 col: 1 fov: 3
1 1 1
1 2 0
Parsing cell masks intensity......Done! 
